In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
%matplotlib inline
import networkx as nx
#import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

In [ ]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

In [ ]:
data = dataset[0]  # Get the first graph object.

print(data)
print('==============================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

In [ ]:
edge_index = data.edge_index
print(edge_index.t())

In [ ]:
from torch_geometric.utils import to_networkx

G = to_networkx(data, to_undirected=True)
visualize_graph(G, color=data.y)

In [ ]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(dataset.num_features, 4)
        self.conv2 = GCNConv(4, 4)
        self.conv3 = GCNConv(4, 2)
        self.classifier = Linear(2, dataset.num_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

In [ ]:
model = GCN()

_, h = model(data.x, data.edge_index)
print(f'Embedding shape: {list(h.shape)}')

visualize_embedding(h, color=data.y)

In [ ]:
def foo(x,y):
    def bar(z):
        return x+y+z
    
    return bar 

a = foo(1,2)
a(3)

In [ ]:
from torch.nn import Linear, Parameter
param = torch.Tensor(4)
b = Parameter(param)
b.data.zero_()

In [ ]:
a = Linear(2,2)
a.reset_parameters()
a

In [ ]:
def mult(x,y):
    return x*y

a = mult
a(1,2)

In [ ]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

In [ ]:
print(x)

In [ ]:
# Feature matrix holding 10 elements with 64 features each:
x = torch.randn(10, 64)

# Assign each element to one of three sets:
index = torch.tensor([0, 0, 1, 0, 2, 0, 2, 1, 0, 2])

output = aggr(x, index)  #  Output shape: [3, 64]

In [ ]:
torch.nn

In [ ]:
from torch_geometric.nn import aggr

# Simple aggregations:
mean_aggr = aggr.MeanAggregation()
max_aggr = aggr.MaxAggregation()

# Feature matrix holding 1000 elements with 64 features each:
x = torch.randn(10, 2)
print('x: ', x)
# Randomly assign elements to 100 sets:
index = torch.randint(0, 4, (10, ))
print('index: ', index)
output = mean_aggr(x, index)  #  Output shape: [100, 64]

print('output: ', output)

In [ ]:
import numpy
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class MyConv(MessagePassing):
    
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        #self.lin = Linear(in_channels, out_channels, bias=False)
        #self.bias = Parameter(torch.Tensor(out_channels))

        #self.reset_parameters()

    #def reset_parameters(self):
    #    self.lin.reset_parameters()
    #    self.bias.data.zero_()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        #edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        #x = self.lin(x)

        # Step 3: Compute normalization.
        #row, col = edge_index
        #deg = degree(col, x.size(0), dtype=x.dtype)
        #deg_inv_sqrt = deg.pow(-0.5)
        #deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        #norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        #out = self.propagate(edge_index, x=x)

        # Step 6: Apply a final bias vector.
        #out += self.bias

        return self.propagate(edge_index, x=x)

    #def message(self, x_j):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
    #    return x_j

conv = MyConv(1, 1)
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

y = conv(x,edge_index)
y

#class GCN(torch.nn.Module):
#    def __init__(self):
#        super().__init__()
#        #torch.manual_seed(1234)
#        self.conv1 = MyConv(1, 4)

#    def forward(self, x, edge_index):
#        h = self.conv1(x, edge_index)
#        return h

#model = GCN()
#print(model)

#for parameter in model.parameters():
#    print(parameter)

In [ ]:
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = Linear(in_channels, out_channels, bias=False)
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        out = self.propagate(edge_index, x=x, norm=norm)

        # Step 6: Apply a final bias vector.
        out += self.bias

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

conv = GCNConv(1, 4)
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

y = conv(x,edge_index)
print(y)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(dataset.num_features, 4)
        self.conv2 = GCNConv(4, 4)
        self.conv3 = GCNConv(4, 2)
        self.classifier = Linear(2, dataset.num_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

for parameter in model.parameters():
    print(parameter)

In [ ]:
import numpy
import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.nn import aggr


class MyConv(MessagePassing):
    
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr=aggr.MedianAggregation())  # "Add" aggregation (Step 5).

    def forward(self, x, edge_index):

        return self.propagate(edge_index, x=x,foo=3,bar=10)
    
    def message(self,x_j,foo,bar):
        print(foo + bar)
        return x_j

conv = MyConv(1, 1)
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[1], [2], [3]], dtype=torch.float)

y = conv(x,edge_index)
y

#class GCN(torch.nn.Module):
#    def __init__(self):
#        super().__init__()
#        #torch.manual_seed(1234)
#        self.conv1 = MyConv(1, 4)

#    def forward(self, x, edge_index):
#        h = self.conv1(x, edge_index)
#        return h

#model = GCN()
#print(model)

#for parameter in model.parameters():
#    print(parameter)

In [ ]:
def mult(x,y=3):
    return x*y 

def foo(x):
    return mult(x,y=2)
foo(1)

In [ ]:
from torch_geometric.nn import aggr
softmax_aggr = aggr.SoftmaxAggregation(learn=True)

# Randomly assign elements to 100 sets:
index = torch.randint(0, 100, (1000, ))

mean_aggr = aggr.MeanAggregation()
print(mean_aggr)
# Feature matrix holding 1000 elements with 64 features each:
x = torch.randn(1000, 64)

output = mean_aggr(x,index)  #  Output shape: [100, 64]
print(x.shape[0])

In [ ]:
from typing import Optional

import torch
from torch import Tensor
from torch.nn import Parameter

from torch_geometric.nn.aggr import Aggregation
from torch_geometric.utils import softmax


class MyMax(Aggregation):
    r"""An aggregation operator that takes the feature-wise maximum across a
    set of elements

    .. math::
        \mathrm{max}(\mathcal{X}) = \max_{\mathbf{x}_i \in \mathcal{X}}
        \mathbf{x}_i.
    """
    def forward(self, x: Tensor, index: Optional[Tensor] = None,
                ptr: Optional[Tensor] = None, dim_size: Optional[int] = None,
                dim: int = -2) -> Tensor:
        index = torch.randint(0,1,(x.shape[0],))
        print(index)
        return self.reduce(x, index, ptr, dim_size, dim, reduce='max')

x = torch.randn(10, 2)
print(x)
out = MyMax()
out(x)

In [ ]:
from typing import Optional

import torch
from torch import Tensor
from torch.nn import Linear, Parameter

from torch_geometric.nn.aggr import Aggregation
from torch_geometric.utils import softmax,scatter, add_self_loops, degree

from torch_geometric.nn import MessagePassing
from torch_geometric.nn import aggr


class MyMax(Aggregation):
    r"""An aggregation operator that takes the feature-wise maximum across a
    set of elements

    .. math::
        \mathrm{max}(\mathcal{X}) = \max_{\mathbf{x}_i \in \mathcal{X}}
        \mathbf{x}_i.
    """
    def forward(self, x: Tensor, index: Optional[Tensor] = None,
                ptr: Optional[Tensor] = None, dim_size: Optional[int] = None,
                dim: int = -2) -> Tensor:
        x_sum = scatter(x,index,reduce='sum')
        new_index = torch.tensor([0,0,1,1])
        dim = x_sum.shape[0]
        out = torch.zeros((dim,2))
        for i in range(dim):
            a = scatter(x_sum[i,:],new_index,reduce='max')
            out[i,:] = a
        return out

#x = torch.randn(10, 2)
#print(x)
#out = MyMax()
#out(x)

class MyConv(MessagePassing):
    
    def __init__(self, in_channels, out_channels):
        #super().__init__(aggr=MyMax())
        super().__init__(aggr='max')
        self.lin = Linear(in_channels,out_channels,bias=False)

    def forward(self, x, edge_index):
        #edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        x = self.lin(x)
        print('lin(x): ', x)
        print('edge_index: ', edge_index)
        out = self.propagate(edge_index, x=x)
        #print('out: ', out)
        return out
    
    def message(self,x_j):
        return x_j

conv = MyConv(2, 4)
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[1,2], [2,3], [5,6]], dtype=torch.float)

y = conv(x,edge_index)
print('y: ', y) 

#class GCN(torch.nn.Module):
#    def __init__(self):
#        super().__init__()
#        #torch.manual_seed(1234)
#        self.conv1 = MyConv(1, 4)

#    def forward(self, x, edge_index):
#        h = self.conv1(x, edge_index)
#        return h

#model = GCN()
#print(model)

#for parameter in model.parameters():
#    print(parameter)

In [ ]:
from torch_geometric.data import Data

from torch_geometric.loader import DataLoader
dataset = []
for i in range(100):
    edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
    x = torch.randn((3,2), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.contiguous())
    dataset.append(data) 

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch.nn as nn

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = MyConv(in_channels, out_channels)
        #self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        print('x: ', x)
        #x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        #x = self.conv2(x, edge_index)

        #return F.log_softmax(x, dim=1)
        return x 
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = GCN().to(device)
#data = dataset[0].to(device)
model = GCN(2,4)
model_synth = GCN(2,4)
train = model_synth(data)
train = train.detach()
print('train: ', train)
#data = dataset[0]
#data = new_data
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

#model.train()
for epoch in range(20):
    optimizer.zero_grad()
    out = model(data)
    #loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    #print('out: ', out)
    #print(out.shape)
    mseloss = nn.MSELoss()
    loss = mseloss(out, train)
    print('LOSS: ', loss)
    loss.backward()
    optimizer.step()

#model.eval()
#pred = model(data).argmax(dim=1)
#correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
#acc = int(correct) / int(data.test_mask.sum())
#print(f'Accuracy: {acc:.4f}')

In [ ]:
#ata
#ataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset[0]
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [ ]:
from torch_geometric.loader import DataLoader

dataset = []
num_nodes = 3
num_graphs = 200
in_channels = 2
out_channels = 4
for i in range(num_graphs):
    edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
    x = torch.randn((num_nodes,in_channels), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.contiguous())
    dataset.append(data) 

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


model = GCN(in_channels,out_channels)
#print('train: ', train)
#data = dataset[0]
#data = new_data
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model_synth = GCN(in_channels,out_channels)

def train():
    model.train()
    for data in train_loader:
        print('data: ', data)
        train_y = model_synth(data)
        train_y = train_y.detach()
        optimizer.zero_grad()
        out = model(data)
        mseloss = nn.MSELoss()
        loss = mseloss(out, train_y)
        print('out_y: ', out)
        print('train_y: ', train_y)
        print('LOSS: ', loss)
        loss.backward()
        optimizer.step()
        
for epoch in range(30):
    train()

for param in model_synth.named_parameters():
    print(param)

for param in model.named_parameters():
    print(param)    
    

In [ ]:
#Learning max product 
from typing import Optional

import torch
from torch import Tensor
from torch.nn import Linear, Parameter

from torch_geometric.nn.aggr import Aggregation
from torch_geometric.utils import softmax,scatter, add_self_loops, degree

from torch_geometric.nn import MessagePassing
from torch_geometric.nn import aggr

#implement custom aggregation 
class MyMax(Aggregation):
    def forward(self, x: Tensor, index: Optional[Tensor] = None,
                ptr: Optional[Tensor] = None, dim_size: Optional[int] = None,
                dim: int = -2) -> Tensor:
        x_sum = scatter(x,index,reduce='sum')
        new_index = torch.tensor([0,0,1,1])
        dim = x_sum.shape[0]
        out = torch.zeros((dim,2))
        for i in range(dim):
            a = scatter(x_sum[i,:],new_index,reduce='max')
            out[i,:] = a
        return out
    
#implement message passing layer 
class MyConv(MessagePassing):
    
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr=MyMax())
        #super().__init__(aggr='max')
        self.lin = Linear(in_channels,out_channels,bias=False)

    def forward(self, x, edge_index):
        x = self.lin(x)
        out = self.propagate(edge_index, x=x)
        return out
    
    def message(self,x_j):
        return x_j

#test that message passing layer is working 
conv = MyConv(2, 4)
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[1,2], [2,3], [5,6]], dtype=torch.float)

y = conv(x,edge_index)
print('y: ', y) 

#generate synthetic dataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
dataset = []
for i in range(100):
    edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
    x = torch.randn((3,2), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.contiguous())
    dataset.append(data) 

#create a dataloader 
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

#create a neural network module that uses MyConv
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch.nn as nn

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = MyConv(in_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        print('x: ', x)
        return x 

#generate data 
from torch_geometric.loader import DataLoader

dataset = []
num_nodes = 3
num_graphs = 200
in_channels = 2
out_channels = 4
for i in range(num_graphs):
    edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
    x = torch.randn((num_nodes,in_channels), dtype=torch.float)
    data = Data(x=x, edge_index=edge_index.contiguous())
    dataset.append(data) 

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


#generate model and optimizer
model = GCN(in_channels,out_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

#synthetic label generation, not the 'y' label in dataset 
model_synth = GCN(in_channels,out_channels)

#training function 
def train():
    model.train()
    for data in train_loader:
        train_y = model_synth(data)
        train_y = train_y.detach()
        optimizer.zero_grad()
        out = model(data)
        mseloss = nn.MSELoss()
        loss = mseloss(out, train_y)
        print('LOSS: ', loss)
        loss.backward()
        optimizer.step()
        
for epoch in range(30):
    train()

#print ground truth parameters
for param in model_synth.named_parameters():
    print(param)

#print learned parameters 
for param in model.named_parameters():
    print(param)    

In [100]:
import torch
from torch.utils.data import Dataset

#input_dim = 4
#output_dim = 1 
#num_samples = 100

class SyntheticDataset(Dataset):
    def __init__(self, num_samples, input_dim, output_dim):
        self.num_samples = num_samples
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.x = torch.randn((num_samples,input_dim))
        #self.A = torch.randn((output_dim,input_dim))
        self.A = torch.eye(input_dim)
        #self.y = torch.matmul(self.A,torch.transpose(self.x,0,1))[0]
        #self.y = torch.matmul(self.A,torch.transpose(self.x,0,1))[0]
        linx = torch.transpose(torch.matmul(self.A,torch.transpose(self.x,0,1)), 0,1)
        values,indices = torch.max(linx,1)
        self.y = values

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Generate random input vector x
        x = self.x[idx,:]

        # Generate random scalar label y
        #y = self.y[idx,:]
        y = self.y[idx]

        return x, y

import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network model
class LinearNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim,bias=False)

    def forward(self, x):
        linx = self.linear(x)
        #print('linx: ', linx)
        y_pred,indices = torch.max(linx,1)
        return y_pred

# Define the dataset and dataloader
ind = 2
outd = 2
dataset = SyntheticDataset(num_samples=1000, input_dim=ind, output_dim=outd)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
#print(dataset[0])


for batch in dataloader:
    inputs, labels = batch
    print('inputs: ', inputs)
    print('labels: ', labels)

#layer = LinearNet(2,2)    
#output = layer(torch.tensor([1,0],dtype=torch.float))
#print(output)

inputs:  tensor([[ 1.0653e+00,  9.7336e-01],
        [ 1.3156e+00,  2.5899e-01],
        [ 1.7049e+00, -8.6683e-01],
        [ 3.0754e-01, -3.7705e-01],
        [-8.5791e-01,  1.6929e+00],
        [ 4.3387e-01, -1.2000e+00],
        [-1.9469e+00,  1.8371e+00],
        [-4.4813e-01,  2.8797e-01],
        [-1.8273e-01, -5.0868e-01],
        [ 6.5906e-01, -4.4661e-01],
        [ 1.0037e+00, -1.7057e+00],
        [-4.0133e-01, -5.7219e-01],
        [ 1.7107e+00, -1.4192e+00],
        [-6.5105e-04,  1.0239e+00],
        [ 4.6422e-01, -1.9570e-01],
        [-6.4466e-01, -3.8710e-01],
        [-5.9740e-01,  3.4725e-01],
        [-1.0449e+00, -5.8654e-01],
        [ 4.1082e-01,  4.0818e-01],
        [ 1.1710e+00,  7.2216e-01],
        [-4.4087e-01, -1.4677e-01],
        [-5.1787e-01,  8.5913e-01],
        [ 3.2008e-01,  2.1867e-01],
        [ 1.8838e+00, -3.0227e-02],
        [-9.1157e-01, -1.4533e-01],
        [-1.8343e-01,  1.0444e+00],
        [-5.8967e-01, -1.2444e+00],
        [ 2.4198e+0

In [101]:
#dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

# Initialize the model, loss function, and optimizer
model = LinearNet(input_dim=ind, output_dim=outd)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Train the model for a fixed number of epochs
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        # Get the inputs and labels from the current batch
        inputs, labels = batch
        #print('inputs: ', inputs)
        #print('labels: ', labels)
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)
        #print('outputs: ', outputs)
        #print('labels: ', labels)
        #print('loss: ', loss)
        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss.item()

    # Print the average loss for the epoch
    epoch_loss = running_loss / len(dataloader)
    print('Epoch {}: Loss = {}'.format(epoch+1, epoch_loss))

for param in model.named_parameters():
    print(param)    

Epoch 1: Loss = 1.0270419269800186
Epoch 2: Loss = 0.5222655832767487
Epoch 3: Loss = 0.2317516206530854
Epoch 4: Loss = 0.09771364042535424
Epoch 5: Loss = 0.0369083670375403
Epoch 6: Loss = 0.012128291607950814
Epoch 7: Loss = 0.0038021491300241905
Epoch 8: Loss = 0.0010127912785264925
Epoch 9: Loss = 0.0002738033222158265
Epoch 10: Loss = 6.523137170688642e-05
('linear.weight', Parameter containing:
tensor([[ 9.9928e-01,  2.5125e-05],
        [-2.7992e-04,  9.9239e-01]], requires_grad=True))


In [34]:
a = torch.randn((10,2))
aT = torch.transpose(a,0,1)
print(aT)
A = torch.randn((1,2))
y = torch.matmul(A,aT)
print(y[0])

tensor([[-0.7424, -1.3996, -0.8878,  0.4946, -0.7376,  0.1015, -0.0164, -0.8289,
         -1.2942,  1.4270],
        [ 0.1913, -0.2592,  0.2095,  0.8133, -0.7425, -0.4972,  0.8100,  1.4298,
          1.6285,  1.2221]])
tensor([-0.0561, -0.6886, -0.0852,  0.9219, -0.9326, -0.4353,  0.7565,  1.0810,
         1.1200,  1.6028])


In [54]:
x = torch.randn((10,2))
#self.A = torch.randn((output_dim,input_dim
A = torch.eye(2)
#self.y = torch.matmul(self.A,torch.transpose(self.x,0,1))[0]
y = torch.transpose(torch.matmul(A,torch.transpose(x,0,1)), 0,1)
values,indices = torch.max(y,1)
print(x)
print(values)

tensor([[-0.8446,  0.7078],
        [-0.8470,  1.2401],
        [-1.2082,  1.2877],
        [-1.5360, -1.1704],
        [-1.4273, -2.3812],
        [ 1.0654, -0.0357],
        [-1.2313, -0.4552],
        [ 0.0124, -0.0129],
        [ 1.1107, -0.8702],
        [-0.2926, -0.7031]])
tensor([ 0.7078,  1.2401,  1.2877, -1.1704, -1.4273,  1.0654, -0.4552,  0.0124,
         1.1107, -0.2926])
